In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import json
import pandas as pd
import numpy as np
from io import StringIO
from collections import defaultdict
from colorama import Fore

In [2]:
lines = []
k = 100000
with open('../data/cut_ab_801_data.txt') as f:
    for i in range(k):
        lines.append(f.readline())

In [3]:
deal_arr = []
order_book_arr = []


for i in range(k):
    io = StringIO(lines[i])
    t = json.load(io)
    if t['type'] == 'OrderBook':
        order_book_arr.append(t)
    else:
        deal_arr.append(t)

        
arr_A = []
arr_B = []
arr_AB = []
for d in order_book_arr:
    if d['symbol'] == 'A':
        arr_A.append(d)
    elif d['symbol'] == 'B':
        arr_B.append(d)
    else:
        arr_AB.append(d)
      
time_hash_A = {}
time_hash_B = {}

for d in arr_A:
    time_hash_A[d['server_time']] = d
    
for d in arr_B:
    time_hash_B[d['server_time']] = d
    
my_time_A = np.array(list(time_hash_A.keys()))
my_time_B = np.array(list(time_hash_B.keys()))
all_time = np.concatenate([my_time_A, my_time_B])
all_time = np.unique(all_time)
all_time = np.sort(all_time)

In [4]:
def print_row(price1, vol1, state1, price2, vol2, state2, sep='\t\t\t'):
    
    if state1 == 'BID':
        color1 = Fore.RED
    elif state1 == 'ASK':
        color1 = Fore.GREEN
    else:
        color1 = Fore.BLACK
        
    if state2 == 'BID':
        color2 = Fore.RED
    elif state2 == 'ASK':
        color2 = Fore.GREEN
    else:
        color2 = Fore.BLACK
    
    print(color1, '{0: <20}'.format(price1), '{0: >7}'.format(vol1), end=sep)
    print(color2, '{0: <20}'.format(price2), '{0: >7}'.format(vol2))

In [5]:
def print_glass(index=0):
    time = all_time[index]
    cur_time_A = my_time_A[np.argmin(my_time_A < time)]
    cur_time_B = my_time_B[np.argmin(my_time_B < time)]
    sep = '\t\t\t'
    print(Fore.BLACK, '{0: <27}'.format(time), sep, '{0: <30}'.format(time))
    print(Fore.BLUE, '{0: <20}'.format(cur_time_A), '{0: >7}'.format('A'), end=sep)
    print(Fore.BLUE, '{0: <20}'.format(cur_time_B), '{0: >7}'.format('B'))
    dict_A = time_hash_A[cur_time_A]
    quotes_ask_A = [t for t in dict_A['quotes'][0]]
    quotes_bid_A = [t for t in dict_A['quotes'][1]]
    
    quotes_vol_dic_A = defaultdict(int)
    quotes_state_dic_A = defaultdict(str)
    for rec in quotes_ask_A:
        quotes_vol_dic_A[rec['price']] = rec['volume']
        quotes_state_dic_A[rec['price']] = 'ASK'
        
    for rec in quotes_bid_A:
        quotes_vol_dic_A[rec['price']] = rec['volume']
        quotes_state_dic_A[rec['price']] = 'BID'
    
    dict_B = time_hash_B[cur_time_B]
    quotes_ask_B = [t for t in dict_B['quotes'][0]]
    quotes_bid_B = [t for t in dict_B['quotes'][1]]
    
    quotes_vol_dic_B = defaultdict(int)
    quotes_state_dic_B = defaultdict(str)
    
    for rec in quotes_ask_B:
        quotes_vol_dic_B[rec['price']] = rec['volume']
        quotes_state_dic_B[rec['price']] = 'ASK'
        
    for rec in quotes_bid_B:
        quotes_vol_dic_B[rec['price']] = rec['volume']
        quotes_state_dic_B[rec['price']] = 'BID'
    step, count = 0.5, 44
    start_A = 8820
    end_A = start_A + step*count
    prices_A = np.arange(start_A, end_A, step)[::-1]
    start_B = 8800
    end_B = start_B + step*count
    prices_B = np.arange(start_B, end_B, step)[::-1]
    for i in range(count):
        vol1, state1 = quotes_vol_dic_A[prices_A[i]], quotes_state_dic_A[prices_A[i]]
        vol2, state2 = quotes_vol_dic_B[prices_B[i]], quotes_state_dic_B[prices_B[i]]
        print_row(prices_A[i], vol1, state1, prices_B[i], vol2, state2)
        

In [6]:
widgets.interact(print_glass,\
                     index=widgets.IntSlider(min=0, max=all_time.shape[0] - 1, step=1))

interactive(children=(IntSlider(value=0, description='index', max=82729), Output()), _dom_classes=('widget-int…

<function __main__.print_glass(index=0)>

In [7]:
deal_arr_A, deal_arr_B = [], []

for d in deal_arr:
    if d['symbol'] == 'A':
        deal_arr_A.append(d)
    elif d['symbol'] == 'B':
        deal_arr_A.append(d)
    else:
        print("oh")

In [9]:
len(deal_arr_A), len(deal_arr_B) #не понятно что делать с этим так как список сделок B пуст

(6494, 0)